# DeepSort Implementation

In [1]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [2]:
cap = cv2.VideoCapture('D:/Image segmentation/Automatic driving test/videos/car_moving.webm')
output_width = 800
output_height = 600
output_file = 'D:/Image segmentation/Automatic driving test/videos/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # For .mp4 files
out = cv2.VideoWriter(output_file, fourcc, 20.0, (output_width, output_height))

model = YOLO("D:/Image segmentation/Automatic driving test/models/yolov8s.pt")  
deepsort = DeepSort(max_age = 50)
object_paths = {}

In [3]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection
    results = model(frame)
    
    # Extract detections
    detections = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Convert tensor to NumPy array and then to list
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().tolist()  # Extract coordinates
            confidence = box.conf[0].cpu().numpy().item()  # Confidence score
            class_id = int(box.cls[0].cpu().numpy().item())  # Class ID
            
            if class_id == 2:  # Only track cars
                # Convert from [x1, y1, x2, y2] to [left, top, width, height]
                left = x1
                top = y1
                width = x2 - x1
                height = y2 - y1
                detections.append(([left, top, width, height], confidence, class_id))

    # Print types and contents for debugging
    print("Type of detections:", type(detections))
    print("Detections:", detections)

    # Update Deep SORT tracker
    tracks = deepsort.update_tracks(detections, frame=frame)

    # Draw bounding boxes, labels, and paths
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()  # Convert to (left, top, right, bottom) format
        x1, y1, x2, y2 = map(int, ltrb)
        
        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f'ID: {track_id}'
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        if track_id not in object_paths:
            object_paths[track_id] = []
        # Center of x and bottom of y
        center_x = int((x1 + x2) / 2)
        bottom_y = int(y2)
        object_paths[track_id].append((center_x, bottom_y))
        
        # Draw the path
        path = object_paths[track_id]
        for i in range(1, len(path)):
            cv2.line(frame, path[i - 1], path[i], (255, 0, 0), 5)

    # Resize the frame
    resized_frame = cv2.resize(frame, (output_width, output_height))

    # Display the frame
    cv2.imshow("Frame", resized_frame)

    # Write the frame to the output video file
    out.write(resized_frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 1 car, 22.0ms
Speed: 3.0ms preprocess, 22.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Type of detections: <class 'list'>
Detections: [([310.1087951660156, 197.47564697265625, 33.081298828125, 34.57987976074219], 0.7794545292854309, 2)]

0: 384x640 1 car, 6.0ms
Speed: 3.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Type of detections: <class 'list'>
Detections: [([310.0439453125, 197.4445037841797, 33.08062744140625, 34.3389892578125], 0.7505561709403992, 2)]

0: 384x640 1 car, 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Type of detections: <class 'list'>
Detections: [([310.1438903808594, 197.34190368652344, 33.3082275390625, 34.41607666015625], 0.7360227704048157, 2)]

0: 384x640 1 car, 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Type of detections: <class 'list'>
Detections: [([310.200561523437